# Problem Statement

## Business Context

In the competitive landscape of retail banking, customer retention is critical for ensuring sustainable growth and profitability. A prominent retail banking institution in Europe provides a range of financial products, including credit cards, loans, and savings accounts, and has been rapidly expanding its customer base across multiple countries. However, with a growing customer base, it faces an increasingly pressing challenge: customer churn. A significant number of customers are closing their accounts and switching to competitors. This decline in customer retention is impacting revenue and long-term customer relationships

Understanding the reasons behind customer attrition (or churn) is essential for the bank to devise effective retention strategies to minimize churn and enhance customer loyalty and satisfaction. The Customer Analytics & Retention Department has been diligently collecting and analyzing historical customer data. Despite the valuable insights provided by historical data, the department grapples with several challenges:

1. **Complex Customer Behavior**: The diverse nature of the bank's offerings and the varying customer preferences across different countries complicate the identification of factors that lead to churn.
2. **Proactive Retention**: The current processes for identifying at-risk customers are reactive rather than proactive, leading to missed opportunities for timely interventions that could prevent churn.

## Objective

The Customer Analytics & Retention Department deployed a churn prediction model as a simple web application to enable real-time, data-driven decision-making through an intuitive interface. However, as access was scaled to geographically distributed teams, the centralized deployment model led to performance bottlenecks and increased latency. Attempts to share the deployment model resulted in frequent failures due to inconsistencies in system environments, operating systems, and dependencies.

To address these challenges, the department aims to find a better mechanism to package the model, environment, dependencies, and configuration into a standardized unit that runs reliably across different systems. The objective is to eliminate compatibility issues, reduce deployment errors, simplify distribution, and allow each location to use the app with minimal setup. Ultimately, it ensures scalable, consistent, and resilient access to the churn prediction system, empowering all teams to take timely, proactive retention actions.

## Solution Approach

The crux of the solution approach is to ***decouple the frontend and backend of the app*** for better accessibility and allowing for easy integration with other services or systems.

1. We’ll first build a Flask app (`app.py`) that loads the serialized ML model and exposes two API endpoints - one for single customer prediction and another for batch predictions using a CSV file.
    - ***This backend handles the prediction logic*** and returns JSON responses.
    - We’ll deploy this Flask app along with the model file and `requirements.txt` into a Hugging Face space using a `Dockerfile`, making it publicly accessible through a URL.

2. Next, we’ll build a separate Streamlit app (`app.py`) that acts as the user interface. This app will include form inputs for online prediction and a CSV uploader for batch prediction.
    - In both cases, it will ***send data to the Flask API*** using the `requests` library and display the responses in a clean, readable format.
    - We’ll deploy this Streamlit app with its own `requirements.txt` to another Hugging Face space.

3. Once both spaces are live, we’ll be able to access the model and make predictions for single as well as multiple users.

# App Backend

## Points to note before executing the below cells
- Go to **Hugging Face**
- Open your **Profile**
- Click on **New Space**
  - Under the space creation, enter the below details
    - Space name: **Backend**
(If you were trying with different names, be cautious when using a underscore `_` in space names, such as `backend_space`, as it can cause exceptions when accessing the API URL. Always use hyphen `-` instead, like `backend-space`.)
    - Select the space SDK: **Docker**
    - Choose a Docker tempplate: **Blank**
    - Click on **Create Space**

## Flask Web Framework


In [ ]:
# Create a folder for storing the files needed for backend server deployment
import os
os.makedirs("backend_files", exist_ok=True)

In [ ]:
%%writefile backend_files/app.py
import joblib
import pandas as pd
from flask import Flask, request, jsonify

# Initialize Flask app with a name
churn_predictor_api = Flask("Customer Churn Predictor")

# Load the trained churn prediction model
model = joblib.load("churn_prediction_model_v1_0.joblib")

# Define a route for the home page
@churn_predictor_api.get('/')
def home():
    return "Welcome to the Customer Churn Prediction API!"

# Define an endpoint to predict churn for a single customer
@churn_predictor_api.post('/v1/customer')
def predict_churn():
    # Get JSON data from the request
    customer_data = request.get_json()

    # Extract relevant customer features from the input data
    sample = {
        'CreditScore': customer_data['CreditScore'],
        'Geography': customer_data['Geography'],
        'Age': customer_data['Age'],
        'Tenure': customer_data['Tenure'],
        'Balance': customer_data['Balance'],
        'NumOfProducts': customer_data['NumOfProducts'],
        'HasCrCard': customer_data['HasCrCard'],
        'IsActiveMember': customer_data['IsActiveMember'],
        'EstimatedSalary': customer_data['EstimatedSalary']
    }

    # Convert the extracted data into a DataFrame
    input_data = pd.DataFrame([sample])

    # Make a churn prediction using the trained model
    prediction = model.predict(input_data).tolist()[0]

    # Map prediction result to a human-readable label
    prediction_label = "churn" if prediction == 1 else "not churn"

    # Return the prediction as a JSON response
    return jsonify({'Prediction': prediction_label})

# Define an endpoint to predict churn for a batch of customers
@churn_predictor_api.post('/v1/customerbatch')
def predict_churn_batch():
    # Get the uploaded CSV file from the request
    file = request.files['file']

    # Read the file into a DataFrame
    input_data = pd.read_csv(file)

    # Make predictions for the batch data and convert raw predictions into a readable format
    predictions = [
        'Churn' if x == 1
        else "Not Churn"
        for x in model.predict(input_data.drop("CustomerId",axis=1)).tolist()
    ]

    cust_id_list = input_data.CustomerId.values.tolist()
    output_dict = dict(zip(cust_id_list, predictions))

    return output_dict

# Run the Flask app in debug mode
if __name__ == '__main__':
    app.run(debug=True)

Writing backend_files/app.py


## Dependencies File

In [ ]:
%%writefile backend_files/requirements.txt
pandas==2.2.2
numpy==2.0.2
scikit-learn==1.6.1
xgboost==2.1.4
joblib==1.4.2
Werkzeug==2.2.2
flask==2.2.2
gunicorn==20.1.0
requests==2.28.1
uvicorn[standard]
streamlit==1.43.2

Writing backend_files/requirements.txt


## Dockerfile

In [ ]:
%%writefile backend_files/Dockerfile
FROM python:3.9-slim

# Set the working directory inside the container
WORKDIR /app

# Copy all files from the current directory to the container's working directory
COPY . .

# Install dependencies from the requirements file without using cache to reduce image size
RUN pip install --no-cache-dir --upgrade -r requirements.txt

# Define the command to start the application using Gunicorn with 4 worker processes
# - `-w 4`: Uses 4 worker processes for handling requests
# - `-b 0.0.0.0:7860`: Binds the server to port 7860 on all network interfaces
# - `app:app`: Runs the Flask app (assuming `app.py` contains the Flask instance named `app`)
CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:7860", "app:churn_predictor_api"]

Writing backend_files/Dockerfile


## Uploading Files to Hugging Face Space for the Backend

**Note**: Before running the code below, ensure that the serialized ML model has been uploaded in to `backend_files` folder.

In [ ]:
# for hugging face space authentication to upload files
from huggingface_hub import login, HfApi

access_key = "----------------"  # Your Hugging Face token created from access keys in write mode
repo_id = "user/space_name"  # Your Hugging Face space id

# Login to Hugging Face platform with the access token
login(token=access_key)

# Initialize the API
api = HfApi()

# Upload Streamlit app files stored in the folder called deployment_files
api.upload_folder(
    folder_path="/content/backend_files",  # Local folder path
    repo_id=repo_id,  # Hugging face space id
    repo_type="space",  # Hugging face repo type "space"
)

# App Frontend

## Points to note before executing the below cells
- Create a Streamlit space on Hugging Face by following the instructions provided on the content page titled **`Creating Spaces and Adding Secrets in Hugging Face`** from Week 1

## Streamlit for Interactive UI

In [ ]:
# Create a folder for storing the files needed for frontend UI deployment
os.makedirs("frontend_files", exist_ok=True)

In [ ]:
%%writefile frontend_files/app.py
import requests
import streamlit as st
import pandas as pd

st.title("Customer Churn Prediction")

# Batch Prediction
st.subheader("Online Prediction")

# Input fields for customer data
CustomerID = st.number_input("Customer ID", min_value=10000000, max_value=99999999)
CreditScore = st.number_input("Credit Score (customer's credit score)", min_value=300, max_value=900, value=650)
Geography = st.selectbox("Geography (country where the customer resides)", ["France", "Germany", "Spain"])
Age = st.number_input("Age (customer's age in years)", min_value=18, max_value=100, value=30)
Tenure = st.number_input("Tenure (number of years the customer has been with the bank)", value=12)
Balance = st.number_input("Account Balance (customer’s account balance)", min_value=0.0, value=10000.0)
NumOfProducts = st.number_input("Number of Products (number of products the customer has with the bank)", min_value=1, value=1)
HasCrCard = st.selectbox("Has Credit Card?", ["Yes", "No"])
IsActiveMember = st.selectbox("Is Active Member?", ["Yes", "No"])
EstimatedSalary = st.number_input("Estimated Salary (customer’s estimated salary)", min_value=0.0, value=50000.0)

customer_data = {
    'CreditScore': CreditScore,
    'Geography': Geography,
    'Age': Age,
    'Tenure': Tenure,
    'Balance': Balance,
    'NumOfProducts': NumOfProducts,
    'HasCrCard': 1 if HasCrCard == "Yes" else 0,
    'IsActiveMember': 1 if IsActiveMember == "Yes" else 0,
    'EstimatedSalary': EstimatedSalary
}

if st.button("Predict", type='primary'):
    response = requests.post("https://<user_name>-<space_name>.hf.space/v1/customer", json=customer_data)    # enter user name and space name before running the cell
    if response.status_code == 200:
        result = response.json()
        churn_prediction = result["Prediction"]  # Extract only the value
        st.write(f"Based on the information provided, the customer with ID {CustomerID} is likely to {churn_prediction}.")
    else:
        st.error("Error in API request")

# Batch Prediction
st.subheader("Batch Prediction")

file = st.file_uploader("Upload CSV file", type=["csv"])
if file is not None:
    if st.button("Predict for Batch", type='primary'):
        response = requests.post("https://<user_name>-<space_name>.hf.space/v1/customerbatch", files={"file": file})    # enter user name and space name before running the cell
        if response.status_code == 200:
            result = response.json()
            st.header("Batch Prediction Results")
            st.write(result)
        else:
            st.error("Error in API request")

Writing frontend_files/app.py


## Dependencies File

In [ ]:
%%writefile frontend_files/requirements.txt
pandas==2.2.2
requests==2.28.1
streamlit==1.43.2

Writing frontend_files/app_requirements.txt


## Dockerfile

In [ ]:
%%writefile frontend_files/Dockerfile
# Use a minimal base image with Python 3.9 installed
FROM python:3.9-slim

# Set the working directory inside the container to /app
WORKDIR /app

# Copy all files from the current directory on the host to the container's /app directory
COPY . .

# Install Python dependencies listed in requirements.txt
RUN pip3 install -r requirements.txt

# Define the command to run the Streamlit app on port 8501 and make it accessible externally
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0", "--server.enableXsrfProtection=false"]

# NOTE: Disable XSRF protection for easier external access in order to make batch predictions

Overwriting frontend_files/Dockerfile


## Uploading Files to Hugging Face Space for the Frontend

In [ ]:
access_key = "-------------"  # Your Hugging Face token created from access keys in write mode
repo_id = "user/space_name"  # Your Hugging Face space id

# Login to Hugging Face platform with the access token
login(token=access_key)

# Initialize the API
api = HfApi()

# Upload Streamlit app files stored in the folder called deployment_files
api.upload_folder(
    folder_path="/content/frontend_files",  # Local folder path
    repo_id=repo_id,  # Hugging face space id
    repo_type="space",  # Hugging face repo type "space"
)

# Inferencing using Flask API


As the ***frontend and backend are decoupled***, we can ***access the backend directly for predictions***.
- The decoupling ensures seamless interaction with the deployed model while leveraging the API for scalable inference.

Let's see how to interact with the Flask API programatically within this notebook to perform **online** and **batch inference**.

We will
1. Send API requests for both online and batch inference.
2. Process and check the model predictions.

In [ ]:
import json  # To handle JSON formatting for API requests and responses
import requests  # To send HTTP requests to the deployed Flask API

import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations

In [ ]:
model_root_url = "https://<user_name>-<space_name>.hf.space/"  # Base URL of the deployed Flask API on Hugging Face Space; enter user name and space name before running the cell

In [ ]:
model_url = model_root_url + "/v1/customer"  # Endpoint for online (single) inference

Since our model predictions are served through the Flask endpoint we created, we need to call this endpoint to make a prediction.

> ```@app.post('/v1/customer')```

In [ ]:
model_batch_url = model_root_url + "/v1/customerbatch"  # Endpoint for batch inference

> ```@app.post('/v1/customerbatch')```

## Online Inference

The idea is to send a single request to the API and receive an immediate response. This is useful for real-time applications like recommendation systems and fraud detection.

* This data is sent as a JSON payload in a POST request to the model endpoint.
* The model processes the input features and returns a prediction as a JSON payload.

In [ ]:
payload = {
    "CreditScore": 900,
    "Geography": "France",
    "Age": 67,
    "Tenure": 1,
    "Balance": 120000.50,
    "NumOfProducts": 1,
    "HasCrCard": 0,
    "IsActiveMember": 0,
    "EstimatedSalary": 95000.75
}

In [ ]:
# Sending a POST request to the model endpoint with the test payload
response = requests.post(model_url, json=payload)

In [ ]:
response

<Response [200]>

In [ ]:
print(response.json())

{'Prediction': 'churn'}


## Batch Inference

The idea is to send a batch of requests to the API and receive a response. The backend reads the entire dataset, runs it through the ML model, and returns the prediction for every row in the file. This is useful for applications like loan default prediction and customer churn prediction, where we don't need results instantaneously.

* This data is sent as a CSV file in a POST request to the model endpoint.
* The model processes each row containing the input features and returns the predictions for each row as one single JSON payload.

In [ ]:
import pandas as pd

In [ ]:
churn_dataset = pd.read_csv("batch_data.csv")

In [ ]:
# List of numerical features in the dataset
numeric_features = [
    'CustomerId',        # Customer unique ID
    'CreditScore',       # Customer's credit score
    'Age',               # Customer's age
    'Tenure',            # Number of years the customer has been with the bank
    'Balance',           # Customer’s account balance
    'NumOfProducts',     # Number of products the customer has with the bank
    'HasCrCard',         # Whether the customer has a credit card (binary: 0 or 1)
    'IsActiveMember',    # Whether the customer is an active member (binary: 0 or 1)
    'EstimatedSalary'    # Customer’s estimated salary
]

# List of categorical features in the dataset
categorical_features = [
    'Geography',         # Country where the customer resides
]

# Define predictor matrix (X) using selected numeric and categorical features
batch_input_data = churn_dataset[numeric_features + categorical_features]

In [ ]:
# Prepare batch input for API request
batch_input = {
    'file': batch_input_data.to_csv(header=True, index=False).encode('utf-8')
}

In [ ]:
# Send request to the model API for batch predictions
response = requests.post(
    model_batch_url,  # Model endpoint URL
    files=batch_input
)

In [ ]:
response

<Response [200]>

In [ ]:
response.text

'{"15574012":"Not Churn","15592389":"Churn","15592531":"Churn","15619304":"Not Churn","15656148":"Not Churn","15701354":"Not Churn","15737173":"Churn","15737888":"Not Churn","15767821":"Churn","15792365":"Not Churn"}\n'

- As we can see, we receive a JSON where each key represents a customer ID, and the value represents the model prediction of whether the customer will churn or not.

<font size=6 color="blue">Power Ahead!</font>
___